Технический отчет



Method | F1 Average |
--- | --- |
rubert-tiny2 (5 epochs) | 0.522 |
rubert-tiny2 (15 epochs, gradient_accumulation_steps = 2, warmup_steps = 500, +label_smoothing = 0.1) | 0.6487 |
rubert-tiny2 (+lr_scheduler_type = polynomial) | 0.653 |
bge-m3-en-ru (+all prev) | **0.78** |
sbert_large_nlu_ru (encoder 6 layer freeze + fp16) | 0.753 |
multilingual-e5-small (without freeze) | 0.63 |

Для текущей задачи ожидаемо будет дообучать предобученные трансформеры, и хотелось в начале проверить улучшения на базовое легкой модели и дальше переходить к более тяжелым.

*   В качестве первой модели рассматривался **rubert_tiny2**,
который добучался as is 5 эпох, в целом было видно что нужно больше обучать тк модель не дообучена. F1 = 0.522

*   Далее бралась та же модель с теми же параметрами, но уже 15 эпох, для стабильности включил аккумуляция градиента=2, в целом лучше брать от 4 и выше, но метрики были хуже, чтобы не сломать предобученную модель добавил разогрев. Значение F1 в этом эксперименте 0.6487.

*   Далее для **rubert_tiny2** добавил polynomial scheduler - что улучшило метрику еще на 1%+, cosine не улучшал. Значение F1 в этом эксперименте 0.653.

*   Далее уже переходим на SOTAу 23года на многих NLP задачах  **bge-m3-en-ru**, vocabulary был сокращен что сделало модель. Все остальные параметры сохранились - получили существенное улучшение в части метрики рост до 0.78

*   Следующим была использована модель **sbert_large_nlu_ru** от Сбера, которая обучалась с фокусом на русский язык. Для ускорения эксперимента были заморожены первые 6 слоев энкодера и обучение велось при fp16. Метрика хуже 0.753, но время обучения уменьшилось в три раза с 30 минут до 10

*   Следующий подход был использована модель **sbert_large_nlu_ru** от Сбера, которая обучалась с фокусом на русский язык. Для ускорения эксперимента были заморожены первые 6 слоев энкодера и обучение велось при fp16. Метрика хуже 0.753, но время обучения уменьшилось в три раза с 30 минут до 10

* Для разнообразия была проверена e5 модель, но большая версия не влезла в ноутбук а small показали низкие метрики  

Выводы:
*   Небольшие модели обучаются быстрее, но требуют больше эпох. rubert_tiny2 за 15 эпох не показал сильного переобучения, тогда как bge-m3-en-ru уже за 6 эпох достиг насыщения.

*   gradient_accumulation_steps, lr_scheduler, label_smoothing и warmup_steps - можно расматривать как некие регуляризаторы, что позволяют улучшать стабильность и качество получаемых моделей. Включение каждого параметра улучшало качество моделей

*   Удивительно, но основной гэп rubert_tiny2 что он не смог обучиться на сущности Finding (F1=0), тогда как более тяжелые модели такого гэпа уже не имели

* **bge-m3** большая модель которая обучалась на 100 языках, и соотвествено доли процента там составляет русских язык. При это его качество сопоставимо и лучше моделей, которые обучались целенаправленно на русских язык **sbert_large_nlu_ru**


## 3.1 Импорт библиотек

In [2]:
!pip install -q --upgrade transformers
!pip install datasets seqeval corus razdel -q
!pip install Partial State
!pip install -q --upgrade datasets
!pip install -U datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 901.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for Partial: filename=partial-1.0-py3-none-any.whl size=1855 sha256=3996a9a2dc41f09c4eb3e1a0225298b83ec25f3f27b623a96112653fb0326a81
  Stored i

In [1]:
from corus import load_rudrec
from razdel import tokenize
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from datasets import Dataset, DatasetDict
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
import transformers
from transformers import pipeline
import logging
from transformers.trainer import logger as noisy_logger

## 3.2 Загрузка данных

In [2]:
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json

--2024-10-05 13:52:48--  https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json [following]
--2024-10-05 13:52:48--  https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1773014 (1.7M) [text/plain]
Saving to: ‘rudrec_annotated.json.1’

rudrec_annotated.js 100%[===================>]   1.69M  --.-KB/s    in 0.05s   

2024-10-05 13:52:48 (35.3 MB/s) - ‘rudrec_annotated.json.1’ saved [1773014/1773014]



In [2]:
drugs = list(load_rudrec('rudrec_annotated.json'))


from collections import Counter, defaultdict
type2text = defaultdict(Counter)
ents = Counter()
for item in drugs:
    for e in item.entities:
        ents[e.entity_type] += 1
        type2text[e.entity_type][e.entity_text] += 1

for k, v in ents.most_common():
    print(k, v)
    print(type2text[k].most_common(3))


def extract_labels(item):
    raw_toks = list(tokenize(item.text))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item.text)
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item.entities:
        e_words = sorted({idx for idx in char2word[e.start:e.end] if idx is not None})
        word_labels[e_words[0]] = 'B-' + e.entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e.entity_type

    return {'tokens': words, 'tags': word_labels}

ner_data = [extract_labels(item) for item in drugs]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list

ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})

def tokenize_and_align_labels(examples, label_all_tokens=False):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


DI 1401
[('простуды', 64), ('ОРВИ', 47), ('профилактики', 42)]
Drugname 1043
[('Виферон', 33), ('Анаферон', 25), ('Циклоферон', 24)]
Drugform 836
[('таблетки', 154), ('таблеток', 79), ('свечи', 63)]
ADR 720
[('аллергия', 16), ('слабость', 13), ('диарея', 12)]
Drugclass 330
[('противовирусный', 21), ('противовирусное', 18), ('противовирусных', 13)]
Finding 236
[('аллергии', 12), ('температуры', 6), ('сонливости', 5)]


## 3.3. Модель

In [3]:
#расчет метрик

def compute_metrics(p):
    predictions, labels, inputs = p.predictions, p.label_ids, p.inputs
    predictions = np.argmax(p.predictions, axis=2)

    # send only the first token of each word to the evaluation
    true_predictions = []
    true_labels = []
    for prediction, label, tokens in zip(predictions, labels, inputs):
        true_predictions.append([])
        true_labels.append([])
        for (p, l, t) in zip(prediction, label, tokens):
            if l != -100 and not tokenizer.convert_ids_to_tokens(int(t)).startswith('##'):
                true_predictions[-1].append(label_list[p])
                true_labels[-1].append(label_list[l])

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
MODELS_DICT = dict()
def add_model(model, name):
  model_dict = dict()
  model_dict['model'] = model
  MODELS_DICT[name] = model_dict

def add_result(result, name):
  if name in MODELS_DICT:
    MODELS_DICT[name]["result"] = result
  else:
    raise BaseException("Нет модели с таким именем в списке")

def add_trainer(trainer, name):
  if name in MODELS_DICT:
    MODELS_DICT[name]["trainer"] = trainer
  else:
    raise BaseException("Нет модели с таким именем в списке")

import evaluate
metric = evaluate.load("seqeval", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [32]:
# model training
batch_size = 20
model_checkpoint = "cointegrated/rubert-tiny2"
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer)

args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=15,
    weight_decay=0.01,
    save_strategy='epoch',
    report_to='none',
    gradient_accumulation_steps = 2,
    warmup_steps=500,
    load_best_model_at_end=True,
    include_inputs_for_metrics=True,
    label_smoothing_factor=0.1,
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

print(model_checkpoint, trainer.evaluate().get('eval_f1'))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/3847 [00:00<?, ? examples/s]

Map:   0%|          | 0/962 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,No log,1.930256,0.021277,0.030655,0.025119,0.812144
2,No log,0.803377,0.644351,0.162791,0.259916,0.911502
4,No log,0.725383,0.500534,0.495772,0.498141,0.934254
6,1.180100,0.694873,0.570825,0.570825,0.570825,0.943636
8,1.180100,0.684251,0.578154,0.649049,0.611554,0.947551
10,0.668100,0.676901,0.621022,0.680761,0.649521,0.951245
12,0.668100,0.675363,0.608007,0.690275,0.646535,0.951540
14,0.668100,0.674270,0.612782,0.689218,0.648756,0.951762


cointegrated/rubert-tiny2 0.6487294469357249


In [33]:
p = trainer.predict(tokenized_datasets["test"])

predictions, labels, inputs = p.predictions, p.label_ids, tokenized_datasets["test"]['input_ids']
predictions = np.argmax(p.predictions, axis=2)

# send only the first token of each word to the evaluation
true_predictions = []
true_labels = []
for prediction, label, tokens in zip(predictions, labels, inputs):
    true_predictions.append([])
    true_labels.append([])
    for (p, l, t) in zip(prediction, label, tokens):
        if l != -100 and not tokenizer.convert_ids_to_tokens(int(t)).startswith('##'):
            true_predictions[-1].append(label_list[p])
            true_labels[-1].append(label_list[l])

results = metric.compute(predictions=true_predictions, references=true_labels)
results

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'ADR': {'precision': 0.2808988764044944,
  'recall': 0.33557046979865773,
  'f1': 0.30581039755351686,
  'number': 149},
 'DI': {'precision': 0.4533678756476684,
  'recall': 0.6481481481481481,
  'f1': 0.5335365853658536,
  'number': 270},
 'Drugclass': {'precision': 0.8970588235294118,
  'recall': 0.8356164383561644,
  'f1': 0.8652482269503545,
  'number': 73},
 'Drugform': {'precision': 0.903954802259887,
  'recall': 0.9302325581395349,
  'f1': 0.9169054441260746,
  'number': 172},
 'Drugname': {'precision': 0.8134920634920635,
  'recall': 0.9070796460176991,
  'f1': 0.8577405857740585,
  'number': 226},
 'Finding': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 56},
 'overall_precision': 0.6135721017907634,
 'overall_recall': 0.6881606765327696,
 'overall_f1': 0.6487294469357249,
 'overall_accuracy': 0.9516879663145453}

**+lr_scheduler_type="polynomial"**

In [6]:
# model training
batch_size = 20
model_checkpoint = "cointegrated/rubert-tiny2"
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer)

args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=15,
    weight_decay=0.01,
    save_strategy='epoch',
    report_to='none',
    gradient_accumulation_steps = 2,
    warmup_steps=500,
    load_best_model_at_end=True,
    include_inputs_for_metrics=True,
    label_smoothing_factor=0.1,
    lr_scheduler_type="polynomial",
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

print(model_checkpoint, trainer.evaluate().get('eval_f1'))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/3847 [00:00<?, ? examples/s]

Map:   0%|          | 0/962 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,No log,2.238304,0.007226,0.045455,0.012469,0.474847
2,No log,0.821966,0.727273,0.042283,0.079920,0.903524
4,No log,0.732214,0.494938,0.465116,0.479564,0.930561
6,1.302800,0.697943,0.568353,0.558140,0.563200,0.941346
8,1.302800,0.685398,0.568053,0.635307,0.599800,0.946369
10,0.671400,0.676879,0.605058,0.657505,0.630193,0.950137
12,0.671400,0.674947,0.606942,0.683932,0.643141,0.952057
14,0.671400,0.673180,0.618868,0.693446,0.654038,0.953313


cointegrated/rubert-tiny2 0.6533266633316659


**bge-m3-en-ru**

In [5]:

# model training
batch_size = 20
model_checkpoint = "qilowoq/bge-m3-en-ru"
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer)

args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=15,
    weight_decay=0.01,
    save_strategy='epoch',
    report_to='none',
    gradient_accumulation_steps = 2,
    warmup_steps=500,
    load_best_model_at_end=True,
    include_inputs_for_metrics=True,
    label_smoothing_factor=0.1,
    lr_scheduler_type="polynomial",
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

print(model_checkpoint, trainer.evaluate().get('eval_f1'))

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at qilowoq/bge-m3-en-ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.32M [00:00<?, ?B/s]

...

Map:   0%|          | 0/3847 [00:00<?, ? examples/s]

Map:   0%|          | 0/962 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,No log,0.820812,0.500000,0.002114,0.004211,0.901086
2,No log,0.631599,0.694686,0.760042,0.725896,0.965133
4,No log,0.620458,0.752212,0.808668,0.779419,0.970377
6,0.821900,0.624537,0.798561,0.821353,0.809797,0.971781
8,0.821900,0.635968,0.761297,0.819239,0.789206,0.967792
10,0.561300,0.633125,0.791374,0.834038,0.812146,0.972003
12,0.561300,0.637864,0.783730,0.835095,0.808598,0.971338
14,0.561300,0.637602,0.784372,0.838266,0.810424,0.971633


qilowoq/bge-m3-en-ru 0.7794192562404483


**ai-forever/sbert_large_nlu_ru with layer freeze + fp16**

In [6]:
from transformers import EarlyStoppingCallback


# model training
batch_size = 20
model_checkpoint = "ai-forever/sbert_large_nlu_ru"
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer)

#freeze some layer in encoder
for name, param in model.bert.named_parameters():
    if "encoder.layer" in name:
        layer_num = int(name.split('.')[2])
        if layer_num < 6:
            param.requires_grad = False

args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=15,
    weight_decay=0.01,
    save_strategy='epoch',
    report_to='none',
    gradient_accumulation_steps = 2,
    warmup_steps=500,
    load_best_model_at_end=True,
    include_inputs_for_metrics=True,
    label_smoothing_factor=0.1,
    lr_scheduler_type="polynomial",
    fp16 = True,
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=2)],
)

trainer.train()

print(model_checkpoint, trainer.evaluate().get('eval_f1'))

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ai-forever/sbert_large_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

...

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/962 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,No log,0.843955,0.000000,0.000000,0.000000,0.900864
2,No log,0.643115,0.653529,0.753700,0.700049,0.960848
4,No log,0.631637,0.739046,0.802326,0.769387,0.968235
6,0.835700,0.636034,0.722948,0.819239,0.768087,0.967349


ai-forever/sbert_large_nlu_ru 0.7535496957403651


**multilingual-e5-small без заморозки**

In [9]:

# model training
batch_size = 20
model_checkpoint = "intfloat/multilingual-e5-small"
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer)

args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=15,
    weight_decay=0.01,
    save_strategy='epoch',
    report_to='none',
    gradient_accumulation_steps = 2,
    warmup_steps=500,
    load_best_model_at_end=True,
    include_inputs_for_metrics=True,
    label_smoothing_factor=0.1,
    lr_scheduler_type="polynomial",
    fp16 = True,
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=2)],
)

trainer.train()

print(model_checkpoint, trainer.evaluate().get('eval_f1'))

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at intfloat/multilingual-e5-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

Map:   0%|          | 0/3847 [00:00<?, ? examples/s]

Map:   0%|          | 0/962 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,No log,2.142389,0.000000,0.000000,0.000000,0.900864
2,No log,1.324570,0.000000,0.000000,0.000000,0.900864
4,No log,1.094154,0.000000,0.000000,0.000000,0.900864
6,1.576700,0.843841,0.426244,0.525370,0.470644,0.934624
8,1.576700,0.752777,0.487156,0.561311,0.521611,0.938391
10,0.814600,0.719217,0.571429,0.604651,0.587571,0.945483
12,0.814600,0.709790,0.598659,0.660677,0.628141,0.949915
14,0.814600,0.704542,0.602305,0.662791,0.631102,0.951245


intfloat/multilingual-e5-small 0.6334519572953737


## 3.4. Применение модели

In [10]:
# model inference
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [12]:
text='нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?'

print(pipe(text))

[{'entity_group': 'DI', 'score': 0.26487315, 'word': '', 'start': 30, 'end': 31}, {'entity_group': 'ADR', 'score': 0.18447974, 'word': 'сыпьюпокрылся,', 'start': 31, 'end': 46}, {'entity_group': 'ADR', 'score': 0.19741158, 'word': 'глазаопухли,сверху', 'start': 46, 'end': 67}, {'entity_group': 'ADR', 'score': 0.21068338, 'word': 'снизунавекахвысыпала', 'start': 69, 'end': 94}, {'entity_group': 'Drugname', 'score': 0.32404393, 'word': 'виферона', 'start': 121, 'end': 130}]


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:398: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(
